In [1]:
import requests
import glob
import os
from pathlib import Path
import re
import json
from general_purpose_agent import GeneralPurposeAgent
from parser.HDSD_excel_to_markdown import process_excel_to_markdown as hdsd_excel_extractor
from parser.docx_extractor import extract_docx_with_images as hdsd_docx_extractor
from parser.qa_docx_extractor import extract_text_and_images as qa_docx_extractor
from parser.qa_xlsx_extractor import process_excel_to_markdown as qa_excel_extractor
import pandas as pd
import pickle

In [3]:
def read_hdsd_excel(file_path):
    with open(file_path, "rb") as f:
        results = hdsd_excel_extractor(file_path, f.read())
    combined_content = []
    for result in results:
        combined_content.append(f"## Sheet {result['sheet_name']}:")
        combined_content.append(result["content"])
        combined_content.append("")  # Thêm dòng trống giữa các sheet
    
    merged_results = {
        "content": "\n".join(combined_content),
        "images": [img for result in results for img in result["images"]]
    }
    return merged_results["content"]


def read_hdsd_docx(file_path):
    return hdsd_docx_extractor(file_path)

def read_qa_docx(file_path):
    with open(file_path, "rb") as f:
        return qa_docx_extractor(file_path, f)['data']

def read_qa_excel(file_path):
    with open(file_path, "rb") as f:
        results = qa_excel_extractor(file_path, f.read())
    combined_content = []
    for result in results:
        combined_content.append(f"## Sheet {result['sheet_name']}:")
        combined_content.append(result["content"])
        combined_content.append("")  # Thêm dòng trống giữa các sheet
    
    merged_results = {
        "content": "\n".join(combined_content),
        "images": [img for result in results for img in result["images"]]
    }
    
    return merged_results['content']

def extract_json(response_text):
    pattern = r"```json\s*([\[{].*?[\]}])\s*```"
    match = re.search(pattern, response_text, re.DOTALL)
    if match:
        return json.loads(match.group(1))
    return None

In [7]:
xlsx_system_prompt = """Bạn là agent để Tạo Cặp Câu Hỏi & Trả Lời (FAQ) để Kiểm Tra Hiệu Năng Hệ Thống Hướng Dẫn Nghiệp Vụ Ngân Hàng

## Mục tiêu
Sinh các cặp FAQ (Frequently Asked Questions) để kiểm thử khả năng hiểu và trả lời đúng của hệ thống hỗ trợ người dùng nghiệp vụ ngân hàng, dựa trên dữ liệu hướng dẫn sử dụng hệ thống ACL - Loan Servicing (Secured Lending).

## Định dạng đầu vào:
- Định dạng đầu vào sẽ là nội dung tài liệu excel:
```
# Sheet 0: Sheet tổng quan
[Nội dung sheet 0]

# Sheet 1
[Nội dung sheet 1]

...
```

## Nhiệm vụ của bạn
Đối với mỗi bước nghiệp vụ (từ các sheet chi tiết):

1. Sinh một cặp FAQ gồm:
   - Câu hỏi: Là phiên bản diễn đạt lại (paraphrased) của nội dung gốc, nhưng vẫn giữ nguyên ý nghĩa
   - Câu trả lời: Giữ nguyên nội dung gốc như trong file

## Định dạng đầu ra
Kết quả trả về phải ở dạng JSON theo cấu trúc sau:

```json
{{
  "index": {{
    "question": "Câu hỏi được tgr",
    "answer": "Câu trả lời giữ nguyên từ file gốc",
    "ref": ["Tên file", "Tên sheet", "STT hoặc dòng liên quan"]
  }},
  ...
}}

## Lưu ý:
- Sheet 0 luôn là sheet tổng qua, QA ví dụ sẽ là quy trình tổng quan (tiêu đề sheet) và các quy trình bên trong nó (nội dung sheet)
- Các sheet khác là sheet cụ thể về bước của từng quy trình con trong quy trình tổng quan
- Các câu hỏi nên là nội dung của các sheet
- Sửa lỗi chính tả tiếng Việt (viết thiếu dấu, các từ liền nhau không cách ra,...)
  - Ví dụ: "Kiểmtra Danh mụchồsơmàNgườigiaodịch" -> "Kiểm tra danh mục chồ sơ mà người giao dịch"
""".strip()

In [8]:
docx_system_prompt = """
# 📄 System Prompt: Trích xuất Q&A từ nội dung `.docx` đã trích xuất hình ảnh

Bạn là một agent chuyên tạo bộ **Q&A (FAQ)** từ các tài liệu `.docx` đã được convert sang text, bao gồm cả hình ảnh được nhúng dưới dạng image markdown `![alt](path)`).

## Instructions:

1. **Bỏ qua các phần không liên quan**:
   - Mục lục dạng liệt kê "Câu 1: …", "Câu 2: …" ở đầu tài liệu.
   - Header/footer lặp lại giữa các trang (nếu còn tồn tại).
   - Các chú thích hành chính (ví dụ: “Tài liệu lưu hành nội bộ”, “Trang X/Y”, …).

2. **Xác định question**:
   - Mỗi khi xuất hiện cụm `"Câu N:"`, nội dung ngay sau dấu `:` chính là **question**.
   - Nếu không có từ khóa `"Câu N:"`, hãy tự động phát hiện câu hỏi dựa trên:
     - Dòng đầu mang tính định hướng ("Làm thế nào", "Khi gặp lỗi", "Trường hợp này xử lý thế nào", v.v.).
     - Các đoạn in đậm, gạch đầu dòng mở đầu cho một tình huống nghiệp vụ cụ thể.
   - Câu hỏi cần rõ nghĩa, không viết tắt, và phù hợp ngữ cảnh nghiệp vụ.

3. **Xác định answer**:
   - Tất cả nội dung nằm **sau câu hỏi đó** cho đến trước câu hỏi kế tiếp (hoặc đến hết file nếu là câu cuối cùng).
   - Nếu có ảnh minh họa, giữ nguyên toàn bộ image tag nằm trong vùng trả lời.

4. **Đánh số thứ tự `<index>`** tương ứng với thứ tự xuất hiện (tự tăng dần từ `1`).

5. **Không tự chế thêm câu hỏi hoặc trả lời ngoài nội dung file.** Chỉ diễn giải lại nếu nội dung gốc quá rời rạc.

## Cấu trúc phản hồi:
```json
{{
  "<index>": {{
    "question": "<nội dung câu hỏi>",
    "answer":  "<nội dung trả lời, giữ nguyên image tag nếu có>"
  }},
  ...
}}
```

## Ví dụ phản hồi:
```json
{{
  "1": {{
    "question": "Khi gặp lỗi 'Không thể xác minh thông tin khách hàng', cần xử lý như thế nào?",
    "answer": "Kiểm tra lại mã số khách hàng hoặc CMND/CCCD. Nếu đúng, liên hệ bộ phận kiểm soát nội bộ để xác minh dữ liệu. <img src=\"data/images/verify_error.png\"></image>"
  }},
  "2": {{
    "question": "Làm thế nào để in hợp đồng tín dụng đã duyệt?",
    "answer": "Truy cập menu 'Hợp đồng' > 'Tra cứu hợp đồng'. Chọn hợp đồng cần in và nhấn 'In hợp đồng'. <img src=\"data/images/print_contract.png\"></image>"
  }}
}}
```

## Lưu ý:
- Câu hỏi - trả lời cần rõ ràng, đúng logic và nghiệp vụ.
- Bảo toàn toàn bộ image tag (không đổi tên, không loại bỏ).
- Format JSON phải hợp lệ để `json.loads(..., ensure_ascii=False)` không báo lỗi.
- Nếu phần trả lời có nhiều bước, trình bày theo thứ tự để dễ hiểu (không cần format bảng).

"""

In [9]:
agent = GeneralPurposeAgent(xlsx_system_prompt)

In [49]:
# QA docx
file_path = Path("data/Q&A_Cam_nang_ACL_lending_23052025.docx")
output_dir = Path(f"output/{file_path.stem}")
os.makedirs(output_dir, exist_ok=True)
data = read_qa_docx(str(file_path))

response = agent.get_response(f"Tạo FAQ giúp tôi, đây là nội dung của tài liệu {file_path.stem}:\n{data}")
with open(output_dir / "response.pkl", "wb") as f:
    pickle.dump(response, f)
json_response = extract_json(response["text"])
with open(output_dir / "json_response.json", "wb") as f:
    json.dump(json_response, f, indent=4, ensure_ascii=False)

KeyboardInterrupt: 

In [50]:
# QA excel
file_path = Path("data/Q&A_Testcase_ACL_ChatBot_v1.xlsx")
output_dir = Path(f"output/{file_path.stem}")
os.makedirs(output_dir, exist_ok=True)
data = read_qa_excel(str(file_path))

response = agent.get_response(f"Tạo FAQ giúp tôi, đây là nội dung của tài liệu Q&A: {file_path.name}:\n{data}")
with open(output_dir / "response.pkl", "wb") as f:
    pickle.dump(response, f)
json_response = extract_json(response["text"])
with open(output_dir / "json_response.json", "wb") as f:
    json.dump(json_response, f, indent=4, ensure_ascii=False)

2025-06-03 00:55:12,637 - INFO - Bắt đầu xử lý nội dung file Excel
2025-06-03 00:55:12,641 - INFO - Đang tải workbook...
2025-06-03 00:55:12,752 - INFO - Đã tải workbook thành công. Số sheet: 1
2025-06-03 00:55:12,753 - INFO - Đang xử lý sheet: Master
2025-06-03 00:55:12,797 - INFO - Sheet Master: 207 hàng, 7 cột có dữ liệu
2025-06-03 00:55:12,801 - INFO - Đang xử lý header của sheet Master
2025-06-03 00:55:12,806 - INFO - Đang xử lý dữ liệu của sheet Master
2025-06-03 00:55:12,834 - INFO - Đang tạo markdown cho sheet Master
2025-06-03 00:55:12,835 - INFO - Đã xử lý xong sheet Master. Số ảnh: 1
2025-06-03 00:55:12,840 - INFO - Đã xử lý xong file Excel. Tổng số sheet: 1


NameError: name 'pickle' is not defined

In [10]:
def process_file(file_path):
    print(f"Processing {file_path}")
    output_dir = Path(f"output/{file_path.stem}")
    os.makedirs(output_dir, exist_ok=True)
    data = read_hdsd_excel(str(file_path))

    response = agent.get_response(f"Tạo FAQ giúp tôi, đây là nội dung của tài liệu HDSD: {file_path.name}:\n{data}")
    with open(output_dir / "response.pkl", "wb") as f:
        pickle.dump(response, f)
    json_response = extract_json(response["text"])
    with open(output_dir / "json_response.json", "w", encoding="utf-8") as f:
        json.dump(json_response, f, indent=4, ensure_ascii=False)
    print(f"Done {file_path}")

# Xử lý tất cả các file .xlsx trong thư mục new_data
input_dir = Path("new_data")
for file_path in input_dir.glob("*.xlsx"):
    process_file(file_path)

Processing new_data/RM - HDSD ACL_KHCN_BSS_nga.ct_No_pic.xlsx


2025-06-11 14:02:00,785 - INFO - Bắt đầu xử lý nội dung file Excel
2025-06-11 14:02:00,785 - INFO - Đang tải workbook...
2025-06-11 14:02:01,794 - INFO - Đã tải workbook thành công. Số sheet: 18
2025-06-11 14:02:01,795 - INFO - Đang xử lý sheet: 0
2025-06-11 14:02:01,798 - INFO - Sheet 0: 26 hàng, 9 cột có dữ liệu
2025-06-11 14:02:01,799 - INFO - Đang xử lý header của sheet 0
2025-06-11 14:02:01,799 - INFO - Đang xử lý dữ liệu của sheet 0
2025-06-11 14:02:01,800 - INFO - Đang tạo markdown cho sheet 0
2025-06-11 14:02:01,800 - INFO - Đã xử lý xong sheet 0. Số ảnh: 0
2025-06-11 14:02:01,801 - INFO - Đang xử lý sheet: 1
2025-06-11 14:02:01,801 - INFO - Sheet 1: 3 hàng, 30 cột có dữ liệu
2025-06-11 14:02:01,801 - INFO - Đang xử lý header của sheet 1
2025-06-11 14:02:01,802 - INFO - Đang xử lý dữ liệu của sheet 1
2025-06-11 14:02:01,802 - INFO - Đang tạo markdown cho sheet 1
2025-06-11 14:02:01,803 - INFO - Đã xử lý xong sheet 1. Số ảnh: 0
2025-06-11 14:02:01,803 - INFO - Đang xử lý sheet: 

image map: 0, dict_keys([])
image map: 2, dict_keys(['R3', 'AD3'])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 1, dict_keys(['D17'])
image map: 0, dict_keys([])
image map: 1, dict_keys(['D28'])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])


2025-06-11 14:02:02,002 - INFO - Đã xử lý xong sheet 10. Số ảnh: 0
2025-06-11 14:02:02,003 - INFO - Đang xử lý sheet: 11
2025-06-11 14:02:02,008 - INFO - Sheet 11: 16 hàng, 4 cột có dữ liệu
2025-06-11 14:02:02,010 - INFO - Đang xử lý header của sheet 11
2025-06-11 14:02:02,011 - INFO - Đang xử lý dữ liệu của sheet 11
2025-06-11 14:02:02,012 - INFO - Đang tạo markdown cho sheet 11
2025-06-11 14:02:02,018 - INFO - Đã xử lý xong sheet 11. Số ảnh: 0
2025-06-11 14:02:02,019 - INFO - Đang xử lý sheet: 12
2025-06-11 14:02:02,028 - INFO - Sheet 12: 18 hàng, 4 cột có dữ liệu
2025-06-11 14:02:02,033 - INFO - Đang xử lý header của sheet 12
2025-06-11 14:02:02,035 - INFO - Đang xử lý dữ liệu của sheet 12
2025-06-11 14:02:02,036 - INFO - Đang tạo markdown cho sheet 12
2025-06-11 14:02:02,040 - INFO - Đã xử lý xong sheet 12. Số ảnh: 0
2025-06-11 14:02:02,041 - INFO - Đang xử lý sheet: 13.1
2025-06-11 14:02:02,047 - INFO - Sheet 13.1: 13 hàng, 4 cột có dữ liệu
2025-06-11 14:02:02,052 - INFO - Đang xử

image map: 0, dict_keys([])
image map: 1, dict_keys(['D17'])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])


...

2025-06-11 14:08:49,749 - INFO - Bắt đầu xử lý nội dung file Excel
2025-06-11 14:08:49,750 - INFO - Đang tải workbook...


Done new_data/VK - HDSD ACL_edit_final.xlsx
Processing new_data/RM - HDSD ACL_KHDN_edit_final.xlsx


2025-06-11 14:08:50,118 - INFO - Đã tải workbook thành công. Số sheet: 17
2025-06-11 14:08:50,118 - INFO - Đang xử lý sheet: 0
2025-06-11 14:08:50,121 - INFO - Sheet 0: 21 hàng, 8 cột có dữ liệu
2025-06-11 14:08:50,125 - INFO - Đang xử lý header của sheet 0
2025-06-11 14:08:50,126 - INFO - Đang xử lý dữ liệu của sheet 0
2025-06-11 14:08:50,127 - INFO - Đang tạo markdown cho sheet 0
2025-06-11 14:08:50,127 - INFO - Đã xử lý xong sheet 0. Số ảnh: 0
2025-06-11 14:08:50,128 - INFO - Đang xử lý sheet: 1
2025-06-11 14:08:50,129 - INFO - Sheet 1: 8 hàng, 3 cột có dữ liệu
2025-06-11 14:08:50,130 - INFO - Đang xử lý header của sheet 1
2025-06-11 14:08:50,131 - INFO - Đang xử lý dữ liệu của sheet 1
2025-06-11 14:08:50,132 - INFO - Đang tạo markdown cho sheet 1
2025-06-11 14:08:50,132 - INFO - Đã xử lý xong sheet 1. Số ảnh: 0
2025-06-11 14:08:50,133 - INFO - Đang xử lý sheet: 2
2025-06-11 14:08:50,134 - INFO - Sheet 2: 11 hàng, 3 cột có dữ liệu
2025-06-11 14:08:50,135 - INFO - Đang xử lý header c

image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])


JSONDecodeError: Invalid control character at: line 49 column 1056 (char 11073)

In [53]:
# HDSD excel
file_path = Path("data/RM - HDSD ACL_KHCN.xlsx")
output_dir = Path(f"output/{file_path.stem}")
os.makedirs(output_dir, exist_ok=True)
data = read_hdsd_excel(str(file_path))

response = agent.get_response(f"Tạo FAQ giúp tôi, đây là nội dung của tài liệu HDSD: {file_path.name}:\n{data}")
with open(output_dir / "response.pkl", "wb") as f:
    pickle.dump(response, f)
json_response = extract_json(response["text"])
with open(output_dir / "json_response.json", "wb") as f:
    json.dump(json_response, f, indent=4, ensure_ascii=False)


2025-06-03 02:06:19,116 - INFO - Bắt đầu xử lý nội dung file Excel
2025-06-03 02:06:19,121 - INFO - Đang tải workbook...
2025-06-03 02:06:19,739 - INFO - Đã tải workbook thành công. Số sheet: 21
2025-06-03 02:06:19,740 - INFO - Đang xử lý sheet: 0
2025-06-03 02:06:19,741 - INFO - Sheet 0: 26 hàng, 8 cột có dữ liệu
2025-06-03 02:06:19,741 - INFO - Đang xử lý header của sheet 0
2025-06-03 02:06:19,742 - INFO - Đang xử lý dữ liệu của sheet 0
2025-06-03 02:06:19,743 - INFO - Đang tạo markdown cho sheet 0
2025-06-03 02:06:19,743 - INFO - Đã xử lý xong sheet 0. Số ảnh: 0
2025-06-03 02:06:19,744 - INFO - Đang xử lý sheet: 1
2025-06-03 02:06:19,745 - INFO - Sheet 1: 9 hàng, 4 cột có dữ liệu
2025-06-03 02:06:19,745 - INFO - Đang xử lý header của sheet 1
2025-06-03 02:06:19,746 - INFO - Đang xử lý dữ liệu của sheet 1
2025-06-03 02:06:19,747 - INFO - Đang tạo markdown cho sheet 1
2025-06-03 02:06:19,751 - INFO - Đã xử lý xong sheet 1. Số ảnh: 0
2025-06-03 02:06:19,752 - INFO - Đang xử lý sheet: 1

TypeError: a bytes-like object is required, not 'str'

In [ ]:
# HDSD docx
agent.change_system_prompt(docx_system_prompt)
file_path = Path("data/2. PL01 - Các quy trình nghiệp vụ của sản phẩm tiền gửi dành cho KH cá nhân (2).docx")
output_dir = Path(f"output/{file_path.stem}")
os.makedirs(output_dir, exist_ok=True)
data = read_hdsd_docx(str(file_path))

response = agent.get_response(f"Tạo FAQ giúp tôi, đây là nội dung của tài liệu HDSD: {file_path.name}:\n{data}")
with open(output_dir / "response.pkl", "wb") as f:
    pickle.dump(response, f)
json_response = extract_json(response["text"])
with open(output_dir / "json_response.json", "wb") as f:
    json.dump(json_response, f, indent=4, ensure_ascii=False)


In [23]:
data_df = {'question': [],
           'answer': [],
           'ref': []}

for index, data in json_response.items():
    data_df['question'].append(data['question'])
    data_df['answer'].append(data['answer'])
    data_df['ref'].append(data['ref'])

In [25]:
df = pd.DataFrame(data_df)
df

,question,answer,ref
0,Làm thế nào để kiểm tra xem khách hàng đã tồn ...,Chọn Cấp tín dụng Bán lẻ/ Cho vay có đảm bảo/ ...,"[Hướng dẫn sử dụng ACL - Loan Servicing, Sheet..."
1,Quy trình tìm kiếm khách hàng đã tồn tại trên ...,Chọn Hambeger menu. Chọn Cấp tín dụng Bán lẻ/Q...,"[Hướng dẫn sử dụng ACL - Loan Servicing, Sheet..."
2,"Khi tạo mới thông tin khách hàng, những điểm n...",Mộ t số tên trường cần lưu ý: Tên khách hàng: ...,"[Hướng dẫn sử dụng ACL - Loan Servicing, Sheet..."
3,Cần nhập những thông tin gì khi tạo mới khách ...,Một số tên trường cần lưu ý: Thông tin quốc tị...,"[Hướng dẫn sử dụng ACL - Loan Servicing, Sheet..."
4,Các bước tạo số tờ trình tín dụng trong hệ thố...,ACL menu/Cho vay có bảo đảm/Tạo mới/Tạo mới kh...,"[Hướng dẫn sử dụng ACL - Loan Servicing, Sheet..."
5,Làm cách nào để tìm kiếm khách hàng khi tạo tờ...,- Chọn Tìm kiềm Khách hàng cà nhân/Khách hàng ...,"[Hướng dẫn sử dụng ACL - Loan Servicing, Sheet..."
6,"Khi tạo tờ trình tín dụng, cần nhập những thôn...","- Nhập các thông tin Sản phẩm vay, Số tiền vay...","[Hướng dẫn sử dụng ACL - Loan Servicing, Sheet..."
7,Làm thế nào để thêm người liên quan vào tờ trì...,Tab Thông tin người liên quan - Để thêm Người ...,"[Hướng dẫn sử dụng ACL - Loan Servicing, Sheet..."
8,Quy trình thêm người đồng vay vào hồ sơ tín dụ...,Tab Người đồng vay - Nếu có Người đồng vay thì...,"[Hướng dẫn sử dụng ACL - Loan Servicing, Sheet..."
9,Cách nhập nguồn thu nhập của khách hàng trong ...,Nhập nguồn trả nợ của KH tại Tab Thu nhập - Bư...,"[Hướng dẫn sử dụng ACL - Loan Servicing, Sheet..."


In [26]:
df.to_excel("data/faq_demo.xlsx", sheet_name="faq")

In [ ]:
data_dir = Path("output/2. PL01 - Các quy trình nghiệp vụ của sản phẩm tiền gửi dành cho KH cá nhân (2)")
data_df = {'question': [],
           'answer': [],
           'ref': []
           }

with open(data_dir / "results.json", "r") as f:
    json_response = json.load(f)

for index, data in json_response.items():
    data_df['question'].append(data['question'])
    data_df['answer'].append(data['answer'])
    # data_df['ref'].append(data['ref'])

print(len(data_df['question']), len(data_df['answer']))
df = pd.DataFrame(data_df)
df.to_excel(data_dir / "faq_demo.xlsx", sheet_name="faq")

13 13


In [12]:
import json
filepath = "output/RM - HDSD ACL_KHCN_BSS_nga.ct_No_pic/json_response.json"
with open(filepath, "r") as f:
    data = json.load(f)
questions = []
true_answers = []
refs = []
for index, item in data.items():
    questions.append(item['question'])
    true_answers.append(item['answer'])
    refs.append(item['ref'])

df = pd.DataFrame({
    'Question': questions,
    'True Answer': true_answers,
    'Refs': refs
})
df.to_excel("output/RM - HDSD ACL_KHCN_BSS_nga.ct_No_pic/RM - HDSD ACL_KHCN_BSS_nga.ct_No_pic_QA.xlsx", index=False)

In [1]:
print("Bạn là một chuyên gia đánh giá chất lượng câu trả lời của chatbot. Hãy đánh giá câu trả lời của agent hỗ trợ ngân hàng VIB so với câu trả lời chuẩn (true answer) dựa trên các tiêu chí sau:\n\n## Tiêu chí đánh giá\n1. Độ bao phủ thông tin (Information Coverage):\n- Điểm 8-10: Câu trả lời bao gồm tất cả thông tin quan trọng có trong true answer, không bỏ sót bất kỳ thông tin nào\n- Điểm 5-7: Câu trả lời bao gồm một phần thông tin quan trọng từ true answer, thiếu một số thông tin\n- Điểm 0-4: Câu trả lời thiếu hầu hết thông tin quan trọng từ true answer\n- Lưu ý: Tiêu chí này chỉ cần xem xét thông tin có trong true answer hay chưa, nếu thừa thì bỏ qua, không cần trừ điểm vì sẽ đánh giá ở tiêu chí khác, bỏ qua cả cách trình bày, đánh số\n\n2. Độ chính xác và liên quan của thông tin (Information Accuracy and Relevance):\n- Điểm 8-10: Thông tin bổ sung (nếu có) đều chính xác, liên quan đến câu hỏi và hỗ trợ người dùng hiểu rõ hơn\n- Điểm 5-7: Thông tin bổ sung (nếu có) chính xác nhưng không hoàn toàn liên quan hoặc có thể gây hiểu nhầm nhẹ\n- Điểm 0-4: Thông tin bổ sung không chính xác, không liên quan đến câu hỏi hoặc gây hiểu sai lệch nghiêm trọng\n\n3. Định dạng và cấu trúc (Format):\n- Điểm 8-10: Câu trả lời được định dạng rõ ràng, có xuống dòng khi tách ý, không có câu diễn giải hoặc hậu tố không cần thiết\n- Điểm 5-7: Câu trả lời có định dạng nhưng chưa tối ưu\n- Điểm 0-4: Câu trả lời không có định dạng rõ ràng\n\n4. Ngôn ngữ và phong cách (Language):\n- Điểm 8-10: Sử dụng ngôn ngữ lịch sự, chuyên nghiệp, phù hợp với ngữ cảnh ngân hàng\n- Điểm 5-7: Ngôn ngữ chấp nhận được nhưng chưa thật sự chuyên nghiệp\n- Điểm 0-4: Ngôn ngữ không phù hợp\n\n5. Xử lý trường hợp không tìm thấy câu trả lời (Handling Unknown):\n- Điểm 8-10: Đề xuất chuyển đến Admin một cách phù hợp khi không tìm thấy câu trả lời\n- Điểm 5-7: Có đề xuất chuyển đến Admin nhưng chưa rõ ràng\n- Điểm 0-4: Không xử lý trường hợp không tìm thấy câu trả lời\n\nHãy đánh giá câu trả lời của agent theo các tiêu chí trên và cho điểm tổng thể từ 0-10.\n\n## Input Format:\n- Câu hỏi: [câu hỏi từ người dùng]\n- True Answer: [câu trả lời chuẩn từ FAQ]\n- Agent Answer: [câu trả lời của agent]\n\n## Output Format:\nĐầu ra của phản hồi phải là một json schema như sau:\n```json\n{{\n  \"information_coverage\": [Điểm số cho độ bao phủ thông tin (Information Coverage)],\n  \"hallucination_control\": [Điểm số kiểm soát Hallucination],\n  \"format_and_structure\": [Điểm số cho dạng và cấu trúc (Format and Structure)],\n  \"language_and_style\": [Điểm số cho ngôn ngữ và phong cách (Language)],\n  \"handling_unknown\": [Điểm số cho xử lý trường hợp không tìm thấy câu trả lời (Handling Unknown)],\n  \"comments\": [Nhận xét, góp ý và cải thiện cho từng tiêu chí điểm]\n}}\n```\n\n## Lưu ý quan trọng:\n- Nếu trong câu trả lời có mô tả cho các bước, hãy bỏ qua, đây không phải điểm trừ\n- Bỏ qua các câu kết thúc như \"Quý khách có cần hỗ trợ gì thêm không\", đây không phải là điểm trừ\n- Đưa ra điểm ngang hàng với tiêu chí đó, ví dụ: Độ bao phủ thông tin (Information Coverage): 10\n- Điểm tổng thể là điểm cuối cùng, không đưa ra cách tính, ví dụ: Điểm tổng thể: 9.8\n- Nhận xét, góp ý và cải thiện cho từng tiêu chí điểm sẽ là json với key là tiêu chí, value là nhận xét, góp ý và cải thiện cho tiêu chí đó")

Bạn là một chuyên gia đánh giá chất lượng câu trả lời của chatbot. Hãy đánh giá câu trả lời của agent hỗ trợ ngân hàng VIB so với câu trả lời chuẩn (true answer) dựa trên các tiêu chí sau:

## Tiêu chí đánh giá
1. Độ bao phủ thông tin (Information Coverage):
- Điểm 8-10: Câu trả lời bao gồm tất cả thông tin quan trọng có trong true answer, không bỏ sót bất kỳ thông tin nào
- Điểm 5-7: Câu trả lời bao gồm một phần thông tin quan trọng từ true answer, thiếu một số thông tin
- Điểm 0-4: Câu trả lời thiếu hầu hết thông tin quan trọng từ true answer
- Lưu ý: Tiêu chí này chỉ cần xem xét thông tin có trong true answer hay chưa, nếu thừa thì bỏ qua, không cần trừ điểm vì sẽ đánh giá ở tiêu chí khác, bỏ qua cả cách trình bày, đánh số

2. Độ chính xác và liên quan của thông tin (Information Accuracy and Relevance):
- Điểm 8-10: Thông tin bổ sung (nếu có) đều chính xác, liên quan đến câu hỏi và hỗ trợ người dùng hiểu rõ hơn
- Điểm 5-7: Thông tin bổ sung (nếu có) chính xác nhưng không hoàn toàn l